In [11]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from datetime import datetime
import time

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)


In [2]:
def get_hotels(string):
    start = string
    url = "https://www.ihg.com/destinations/us/en/united-states-hotels"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    states = soup.find('div',{'class' : 'countryListingContainer col-xs-12'})

    states_list =[]
    for x in states.find_all('li'):
        state = {}
        state['name'] = x.find('a').text.strip()[:-7]
        state['url'] = x.find('a').attrs['href']
        states_list.append(state)
    states_df = pd.DataFrame(states_list)
    states_df1 = states_df
        
    cities_list =[]
    counter = 0
    for index, row in states_df1.iterrows():
        state_name = row['name']
        state_url = row['url']
        state_res = requests.get(state_url)
        state_soup = BeautifulSoup(state_res.content, 'lxml')
        cities = state_soup.find('div',{'class' : 'countryListingContainer col-xs-12'})
        for x in cities.find_all('li'):
            city = {}
            city['name'] = x.find('a').text.strip()[:-7]
            city['url'] = x.find('a').attrs['href']
            cities_list.append(city)
        if (counter + 1) % 1 == 0:
            print(f"Completed {state_name} at {datetime.now()}.")
        counter += 1
    cities_df = pd.DataFrame(cities_list)
#     return cities_df

# -----------------------------------------------------------------------------------------

    counter = 0
    items_list = []
    for index, row in cities_df.iterrows(): 
        city = row['name']
        city_url = row['url']
    #         city = city.lower()
    #         city = city.replace(' ','-')
    #         url = 'https://www.ihg.com/destinations/us/en/united-states/'
    #         state_url = url + state1

        try:
            city_url = city_url
    #             city_url = 'https://www.ihg.com/destinations/us/en/united-states/alabama/alabaster-hotels'
            res = requests.get(city_url)
    #             time.sleep(2)
            soup = BeautifulSoup(res.content, 'lxml')
            hotel_soup = soup.find('div', {'class' : "row row-hotel-list"})
    #             print(hotel_soup)
            for x in hotel_soup.find_all('p'):
                for z in x.find_all('a'):
    #                     print(z)
                    items = {}
                    items['name'] = z.text
                    items['url'] = z.attrs['href']
                    if items not in items_list:
                        items_list.append(items)
                    else:
                        pass 

        except:
            pass
        if (counter + 1) % 1 == 0:
            print(f"Completed {city} at {datetime.now()}.")
        counter += 1
    #     return items_list
    dataframe = pd.DataFrame(items_list)
    dataframe = dataframe.drop_duplicates()    

#     new_list = []
#     for index, row in dataframe.iterrows():
#         name = row['name']
#         url = row['url']
#         xyz = {}
#         if state1 in url:
#             xyz['name'] = name
#             xyz['url'] = url
#             new_list.append(xyz)
#     dataframe = pd.DataFrame(new_list)
    print('='*40)
    print(f"The shape of the hotel dataframe is {dataframe.shape}.")
    print('='*40)
    dataframe.to_csv('ihg_urls.csv')

# ----------------------------------------------------------------------------- 
    dataframe = dataframe
    descriptions = []
    addresses = []
    ratings = []
    missing = []
    counter = 0
    for index, row in dataframe.iterrows():
        name = row['name']
        url = row['url']
#         print(name)
#         print(url)
        try: # FOR HOLIDAY INN EXPRESS
            res = requests.get(url)
            soup = BeautifulSoup(res.content, 'lxml')
            itemaddress = soup.find('div',{'itemprop' : 'address'})
            for x in itemaddress.find_all('p'):
                address = {}
                address['name'] = name
                address['url'] = url
                address['street'] = x.find_all('span')[0].text.strip()
                address['locality'] = x.find_all('span')[1].text.strip()
                address['region'] = x.find_all('span')[2].text.strip()
                address['postalcode'] = x.find_all('span')[3].text.strip()
                addresses.append(address)
            itemdescription = soup.find_all('div', {'class' : "wl-txt-sec"})
            for x in itemdescription:
                description = {}
                description['name'] = name
                description['description'] = x.text
                descriptions.append(description)
                itemrating = soup.find('div',{'class' : 'reviews responsiveReviews'})
                rating = {}
                rating['name'] = name
                rating['rating'] = itemrating.find_all('span')[1].text
                ratings.append(rating)
        except:
            try: # FOR HOLIDAY INN
                res = requests.get(url)
                soup = BeautifulSoup(res.content, 'lxml')
                itemaddress = soup.find('div',{'itemprop' : 'address'})
                fulladdress = itemaddress.find_all('p')
                address = {}
                address['name'] = name
                address['url'] = url
                address['street'] = fulladdress[0].text.strip()
                city_state = fulladdress[4].text.strip().split(',')
                address['locality'] =city_state[0]
                address['region']= city_state[1].replace('\xa0','')
                address['postalcode'] = fulladdress[5].text.strip()
                addresses.append(address)
                itemdescription = soup.find_all('div', {'class' : "wl-txt-sec"})
                for x in itemdescription:
                    description = {}
                    description['name'] = name
                    description['description'] = x.text
                    descriptions.append(description)
                itemrating = soup.find('div',{'class' : 'reviews responsiveReviews'})
                rating = {}
                rating['name'] = name
                rating['rating'] = itemrating.find_all('span')[1].text
                ratings.append(rating)
                
            except:
                try: # FOR CANDLEWOOD SUITES
                    res = requests.get(url)
                    soup = BeautifulSoup(res.content, 'lxml')
                    itemaddress = soup.find('p',{'itemprop' : 'address'})
                    fulladdress = itemaddress.find_all('span')
                    address = {}
                    address['name'] = name
                    address['url'] = url
                    address['street'] = fulladdress[0].text.strip()
                    address['locality'] =fulladdress[1].text.strip()
                    address3 = fulladdress[2].text.strip()
                    address['region']= address3[:-20]
                    address['postalcode'] = address3[-19:-14]
                    address['address3'] = address3
                    addresses.append(address)
                    itemdescription = soup.find_all('div', {'class' : "wl-txt-sec"})
                    for x in itemdescription:
                        description = {}
                        description['name'] = name
                        description['description'] = x.text
                        descriptions.append(description)
                    itemrating = soup2.find('div',{'class' : 'show-rating'})
                    rating = {}
                    rating['name'] = name
                    rating['rating'] = itemrating.find_all('span')[0].text
                    ratings.append(rating)
                except:
                    try: # FOR HOTEL INDIGO
                        res = requests.get(url)
                        soup = BeautifulSoup(res.content, 'lxml')                        
                        itemaddress = soup.find('span',{'itemprop' : 'address'})
                        fulladdress = itemaddress.find_all('span')
                        address = {}
                        address['name'] = name
                        address['url'] = url
                        address['street'] = fulladdress[0].text.strip()
                        address['locality'] =fulladdress[1].text.strip()
                        address['region'] =fulladdress[2].text.strip()
                        address['postalcode'] =fulladdress[3].text.strip()
                        addresses.append(address)
                        description = {}
                        full_description = ""
                        descriptionsoup = soup.find('div', {'class' : "desc"})
                        for x in descriptionsoup('p'):
                            x = x.text
                            x = " " + x
                            full_description += (x)
                        extra_description = soup.find('div', {'class' : "in-highlight"})
                        for x in extra_description('div', {'class' : 'highlight-desc'}):
                            x = x.text.strip()
                            x = " " + x
                            full_description += x
                        full_description = full_description.replace('\n','').replace('\t','').replace('\xa0', '').replace('Learn More', '').strip()
                        description['name'] = name
                        description['description'] = full_description
                        descriptions.append(description)
                        itemrating = soup.find('div',{'class' : 'reviews responsiveReviews'})
                        rating = {}
                        rating['name'] = name
                        rating['rating'] = itemrating.find_all('span')[1].text
                        ratings.append(rating)
                        print(address)
                        print(description)
                        print(rating)
                        print()
                        print('='*50)
                        print(counter)
                        print()

                    except:
                        try: # For Kimpton Hotels
                            res = requests.get(url)
                            soup = BeautifulSoup(res.content, 'lxml')
                            itemaddress = soup.find('div',{'class' : 'desc'})
                            itemaddress = str(itemaddress).split('<br/>')
                            newaddress = []
                            address = {}
                            for x in itemaddress:
                                x = x.replace('<div class="desc">\n<span class="visible-content">','').replace('</span>','')
                                newaddress.append(x)
                            address['name'] = name
                            address['url'] = url
                            address['street'] = newaddress[0]
                            address['locality'] = newaddress[1][:-8]
                            address['region'] = newaddress[1][-8:-5]
                            address['postalcode'] = newaddress[1][-5:]
                            addresses.append(address)
                            descriptionsoup = soup.find_all('div',{'class' : 'desc'})
                            description = {}
                            desc = ''
                            for x in descriptionsoup[1:]:
                                x = x.text.strip()
                                x = x.replace('Learn More', '').replace('\t','').replace('\n','')
                                x = " " + x
                                desc += x
                            description['name'] = name
                            description['description'] = desc
                            descriptions.append(description)
                            print(address)
                            print(description)
            #                 print(rating)
                            print()
                            print('='*50)
                            print(counter)
                            print()

                        except:
                            try: # For Crowne Plaza
                                res = requests.get(url)
                                soup = BeautifulSoup(res.content, 'lxml')                                
                                itemaddress = soup.find('div',{'class' : 'new-hinfo-address'})
                                hoteladdress = {}
                                newaddress = []
                                for x in itemaddress.find_all('p'):
                                    address = x.find_all('a')[0].text
                                    address = address.split('|')
                                    for x in address:
                                        x = x.replace('\n','').replace('\t','')
                                        newaddress.append(x)
                                newaddress[0] = newaddress[0].split(',')
                                # address['name'] = name
                                # address['url'] = url
                                hoteladdress['street'] = newaddress[0][0].strip()
                                hoteladdress['locality'] = newaddress[0][1].strip()
                                hoteladdress['region'] = newaddress[1].strip()
                                hoteladdress['postalcode'] = newaddress[2].strip()
                                addresses.append(hoteladdress)
                                itemdescription = soup.find_all('div', {'class' : 'wl-txt-sec'})
                                description = {}
                                description['name'] = name
                                description['description'] = itemdescription[0].text
                                descriptions.append(description)
                                itemrating = soup.find('div', {'class' : 'reviews responsiveReviews'})
                                rating = {}
                                rating['name'] = name
                                rating['rating'] = itemrating.find_all('p')[0].text
                                ratings.append(rating)
                                print(address)
                                print(description)
                                print(rating)
                                print()
                                print('='*50)
                                print(counter)
                                print()

                            except:
                                try: # for InterContinental
                                    res = requests.get(url)
                                    soup = BeautifulSoup(res.content, 'lxml')        
                                    itemrating8 = soup.find_all('div',{'class' : 'address'})    
                                    address = {}
                                    address = []
                                    for x in itemrating8:
                                        x = x.find_all('span')[0].text
                                        x = x.split(',')
                                        for x in x:
                                            address.append(x)
                                    address['name'] = name
                                    address['url'] = url
                                    hoteladdress['street'] = address[0].split(':')[0].strip()
                                    hoteladdress['locality'] = address[1].strip()
                                    hoteladdress['region'] = address[0].split(':')[1].strip()
                                    hoteladdress['postalcode'] = address[2].strip()
                                    addresses.append(address)
                                    itemrating9 = soup.find_all('div',{'class' : 'hotel-description'})
                                    descriptions = {}
                                    description['name'] = name
                                    description['description'] = itemrating9[0].text
                                    descriptions.append(description)
                                    print(address)
                                    print(description)
            #                         print(rating)
                                    print()
                                    print('='*50)
                                    print(counter)
                                    print()
                                except:
                                    description = {}
                                    description['name'] = name
                                    description['url'] = url
                                    missing.append(description)
                                    
        if (counter + 1) % 1 == 0:
            print(f"Attempted to collect {counter} hotel descriptions at {datetime.now()}.")
        counter += 1
    missing_df = pd.DataFrame(missing)
    descriptions_df = pd.DataFrame(descriptions)
    ratings_df = pd.DataFrame(ratings)
    addresses_df = pd.DataFrame(addresses)
#     geotags_df = pd.DataFrame(geotags)
    hotels_df = descriptions_df.merge(addresses_df, on='name')
    hotels_df = hotels_df.merge(ratings_df, on='name')
    hotels_df = hotels_df.drop_duplicates()
    return hotels_df, missing_df

In [3]:
%%time

hotels_df, missing_df = get_hotels('start')

Completed Alabama at 2020-09-25 18:24:46.354739.
Completed Alaska at 2020-09-25 18:24:46.516952.
Completed Arizona at 2020-09-25 18:24:46.671646.
Completed Arkansas at 2020-09-25 18:24:46.989697.
Completed California at 2020-09-25 18:24:47.198645.
Completed Colorado at 2020-09-25 18:24:47.385750.
Completed Connecticut at 2020-09-25 18:24:47.549645.
Completed Delaware at 2020-09-25 18:24:47.701756.
Completed District Of Columbia at 2020-09-25 18:24:47.883757.
Completed Florida at 2020-09-25 18:24:48.092602.
Completed Georgia at 2020-09-25 18:24:48.396843.
Completed Hawaii at 2020-09-25 18:24:48.637714.
Completed Idaho at 2020-09-25 18:24:48.982317.
Completed Illinois at 2020-09-25 18:24:49.275437.
Completed Indiana at 2020-09-25 18:24:49.654563.
Completed Iowa at 2020-09-25 18:24:49.820589.
Completed Kansas at 2020-09-25 18:24:49.985832.
Completed Kentucky at 2020-09-25 18:24:50.150616.
Completed Louisiana at 2020-09-25 18:24:50.311203.
Completed Maine at 2020-09-25 18:24:50.466371.
Com

Completed Anaheim at 2020-09-25 18:25:32.664225.
Completed Atascadero at 2020-09-25 18:25:32.867172.
Completed Auburn at 2020-09-25 18:25:33.102336.
Completed Avalon at 2020-09-25 18:25:33.330035.
Completed Bakersfield at 2020-09-25 18:25:33.527019.
Completed Banning at 2020-09-25 18:25:34.003860.
Completed Barstow at 2020-09-25 18:25:34.238250.
Completed Beaumont at 2020-09-25 18:25:34.782059.
Completed Belmont at 2020-09-25 18:25:35.617194.
Completed Benicia at 2020-09-25 18:25:36.744742.
Completed Berkeley at 2020-09-25 18:25:37.570996.
Completed Big Bear Lake at 2020-09-25 18:25:38.018257.
Completed Bishop at 2020-09-25 18:25:38.233347.
Completed Blythe at 2020-09-25 18:25:38.425242.
Completed Brentwood at 2020-09-25 18:25:38.930940.
Completed Buena Park at 2020-09-25 18:25:40.389790.
Completed Burbank at 2020-09-25 18:25:41.980554.
Completed Burlingame at 2020-09-25 18:25:45.019993.
Completed Calexico at 2020-09-25 18:25:45.802818.
Completed Camarillo at 2020-09-25 18:25:46.243227

Completed Victorville at 2020-09-25 18:26:59.825541.
Completed Visalia at 2020-09-25 18:27:00.072221.
Completed Walnut Creek at 2020-09-25 18:27:00.380242.
Completed Watsonville at 2020-09-25 18:27:01.081279.
Completed West Covina at 2020-09-25 18:27:01.632426.
Completed West Hollywood at 2020-09-25 18:27:03.251690.
Completed West Los Angeles at 2020-09-25 18:27:04.423045.
Completed West Sacramento at 2020-09-25 18:27:04.753552.
Completed Westley at 2020-09-25 18:27:05.590333.
Completed Willows at 2020-09-25 18:27:06.039608.
Completed Windsor at 2020-09-25 18:27:06.457684.
Completed Woodland at 2020-09-25 18:27:06.902457.
Completed Woodland Hills at 2020-09-25 18:27:07.350200.
Completed Yreka at 2020-09-25 18:27:07.529001.
Completed Alamosa at 2020-09-25 18:27:07.715727.
Completed Aurora at 2020-09-25 18:27:08.038248.
Completed Boulder at 2020-09-25 18:27:08.342612.
Completed Brighton at 2020-09-25 18:27:08.744622.
Completed Broomfield at 2020-09-25 18:27:09.059272.
Completed Canon Cit

Completed Panama City Beach at 2020-09-25 18:28:36.747844.
Completed Pembroke Pines at 2020-09-25 18:28:37.100776.
Completed Pensacola at 2020-09-25 18:28:37.330136.
Completed Pensacola Beach at 2020-09-25 18:28:37.642792.
Completed Perry at 2020-09-25 18:28:37.813468.
Completed Plant City at 2020-09-25 18:28:38.120085.
Completed Plantation at 2020-09-25 18:28:38.493892.
Completed Port Charlotte at 2020-09-25 18:28:38.707706.
Completed Port Richey at 2020-09-25 18:28:39.094842.
Completed Port St. Lucie at 2020-09-25 18:28:39.759247.
Completed Punta Gorda at 2020-09-25 18:28:40.584262.
Completed Quincy at 2020-09-25 18:28:41.911868.
Completed Ruskin at 2020-09-25 18:28:42.217808.
Completed Safety Harbor at 2020-09-25 18:28:42.638644.
Completed Saint Augustine at 2020-09-25 18:28:42.881496.
Completed Sanford at 2020-09-25 18:28:43.151259.
Completed Sarasota at 2020-09-25 18:28:43.372913.
Completed Sebring at 2020-09-25 18:28:43.562749.
Completed Silver Springs at 2020-09-25 18:28:43.7585

Completed Galesburg at 2020-09-25 18:29:52.388951.
Completed Glen Ellyn at 2020-09-25 18:29:52.826442.
Completed Glenview at 2020-09-25 18:29:53.429508.
Completed Gurnee at 2020-09-25 18:29:54.030291.
Completed Hillside at 2020-09-25 18:29:54.756340.
Completed Hoffman Estates at 2020-09-25 18:29:55.491400.
Completed Joliet at 2020-09-25 18:29:55.998638.
Completed Lake Zurich at 2020-09-25 18:29:56.595205.
Completed Lansing at 2020-09-25 18:29:57.008118.
Completed Le Roy at 2020-09-25 18:29:57.244394.
Completed Libertyville at 2020-09-25 18:29:57.627251.
Completed Lincolnshire at 2020-09-25 18:29:58.306500.
Completed Litchfield at 2020-09-25 18:29:58.624649.
Completed Loves Park at 2020-09-25 18:29:58.965829.
Completed Marion at 2020-09-25 18:29:59.700464.
Completed Matteson at 2020-09-25 18:30:00.045508.
Completed Mattoon at 2020-09-25 18:30:00.334740.
Completed Metropolis at 2020-09-25 18:30:01.101379.
Completed Mokena at 2020-09-25 18:30:02.501759.
Completed Moline at 2020-09-25 18:3

...

Attempted to collect 3796 hotel descriptions at 2020-09-25 19:30:37.211989.
Attempted to collect 3797 hotel descriptions at 2020-09-25 19:30:37.447582.
Attempted to collect 3798 hotel descriptions at 2020-09-25 19:30:37.933357.
Attempted to collect 3799 hotel descriptions at 2020-09-25 19:30:38.160155.
Attempted to collect 3800 hotel descriptions at 2020-09-25 19:30:38.433746.
Attempted to collect 3801 hotel descriptions at 2020-09-25 19:30:38.750176.
Attempted to collect 3802 hotel descriptions at 2020-09-25 19:30:40.560372.
Attempted to collect 3803 hotel descriptions at 2020-09-25 19:30:40.886550.
['\n\t\t\t\t\t\t\t\t\t17338 International Blvd,   Seattle ', '  Washington ', '  98188 ', '  United States ', '']
{'name': 'Crowne Plaza Seattle Airport', 'description': 'Welcome to the Crowne Plaza Seattle Airport Located across the sky bridge from Seattle Tacoma International Airport and adjacent to the train station, where the light rail train leaves every 12 minutes to downtown Seattle

{'name': 'Kimpton Hotel Vintage Seattle', 'url': 'https://www.ihg.com/kimptonhotels/hotels/us/en/hotel-vintage-seattle-wa/seavp/hoteldetail', 'street': '1100 Fifth Avenue', 'locality': 'Seattle ', 'region': 'WA ', 'postalcode': '98101'}
{'name': 'Kimpton Hotel Vintage Seattle', 'description': " Kimpton Hotel Vintage Seattle is Suspending Operations  We’re committed to keeping our employees, guests, their families and our communities safe during the COVID-19 pandemic. With that, we have a collective responsibility on social distancing and have made the difficult decision to temporarily suspend our operations.  Know that this isn’t goodbye. Travel will thrive again soon and when it does, we look forward to celebrating all of life’s happy occasions with you. Smart, stylish and centrally located, Kimpton Hotel Vintage Seattle has everything you want for your Emerald City trip. With a convenient downtown setting located on Fifth Avenue, you’re just a salmon toss away from Pike Place Market,

Attempted to collect 3817 hotel descriptions at 2020-09-25 19:31:03.110568.
Attempted to collect 3818 hotel descriptions at 2020-09-25 19:31:05.445703.
Attempted to collect 3819 hotel descriptions at 2020-09-25 19:31:07.817165.
Attempted to collect 3820 hotel descriptions at 2020-09-25 19:31:10.363276.
Attempted to collect 3821 hotel descriptions at 2020-09-25 19:31:10.769094.
Attempted to collect 3822 hotel descriptions at 2020-09-25 19:31:11.039308.
Attempted to collect 3823 hotel descriptions at 2020-09-25 19:31:11.638584.
Attempted to collect 3824 hotel descriptions at 2020-09-25 19:31:11.921520.
Attempted to collect 3825 hotel descriptions at 2020-09-25 19:31:15.213897.
Attempted to collect 3826 hotel descriptions at 2020-09-25 19:31:16.160405.
Attempted to collect 3827 hotel descriptions at 2020-09-25 19:31:16.508946.
Attempted to collect 3828 hotel descriptions at 2020-09-25 19:31:17.443780.
Attempted to collect 3829 hotel descriptions at 2020-09-25 19:31:17.833730.
Attempted to

Attempted to collect 3883 hotel descriptions at 2020-09-25 19:31:52.985135.
Attempted to collect 3884 hotel descriptions at 2020-09-25 19:31:53.606211.
Attempted to collect 3885 hotel descriptions at 2020-09-25 19:31:53.929577.
Attempted to collect 3886 hotel descriptions at 2020-09-25 19:31:54.232270.
Attempted to collect 3887 hotel descriptions at 2020-09-25 19:31:54.853345.
{'name': 'Kimpton Journeyman', 'url': 'https://www.ihg.com/kimptonhotels/hotels/us/en/journeyman-hotel-milwaukee-wi/mketj/hoteldetail', 'street': '310 E. Chicago Street', 'locality': 'Milwaukee ', 'region': 'WI ', 'postalcode': '53202'}
{'name': 'Kimpton Journeyman', 'description': " Like a journeyman who’s picked up great stories along the way, The Kimpton Journeyman Hotel is full of fun things to discover — like ceramic pooches in the rooms. Head to rooftop restaurant and bar The Outsider for a game of shuffleboard, or to Mediterranean-inspired modern American restaurant Tre Rivali for a plate of pasta. Any tim

Attempted to collect 3904 hotel descriptions at 2020-09-25 19:32:15.590911.
Attempted to collect 3905 hotel descriptions at 2020-09-25 19:32:16.433034.
Attempted to collect 3906 hotel descriptions at 2020-09-25 19:32:16.829233.
Attempted to collect 3907 hotel descriptions at 2020-09-25 19:32:17.620098.
Attempted to collect 3908 hotel descriptions at 2020-09-25 19:32:18.500052.
Attempted to collect 3909 hotel descriptions at 2020-09-25 19:32:21.699125.
Attempted to collect 3910 hotel descriptions at 2020-09-25 19:32:22.006781.
Attempted to collect 3911 hotel descriptions at 2020-09-25 19:32:22.287515.
Attempted to collect 3912 hotel descriptions at 2020-09-25 19:32:22.725832.
Attempted to collect 3913 hotel descriptions at 2020-09-25 19:32:23.072129.
Attempted to collect 3914 hotel descriptions at 2020-09-25 19:32:23.380663.
Attempted to collect 3915 hotel descriptions at 2020-09-25 19:32:25.023880.
Attempted to collect 3916 hotel descriptions at 2020-09-25 19:32:25.547106.
Attempted to

In [12]:
hotels_df.to_csv('ihg_hotels_descriptions2.csv')

In [13]:
missing_df.to_csv('missing_ihg_hotels2.csv')

In [4]:
hotels_df

,name,description,url,street,locality,region,postalcode,address3,rating
0,Holiday Inn Express & Suites Alabaster,"The Holiday Inn Express & Suites, Alabaster in...",https://www.ihg.com/holidayinnexpress/hotels/u...,"1000 Balmoral Drive ,","Alabaster,",Alabama,35007,NaN,4.7
1,Holiday Inn Express & Suites Birmingham South ...,Nestled at the foot of Alabama's largest state...,https://www.ihg.com/holidayinnexpress/hotels/u...,"260 Cahaba Valley Road ,","Pelham,",Alabama,35124,NaN,4.6
2,Holiday Inn Birmingham - Hoover,"Welcome to Holiday Inn Hoover, Birmingham/Hoov...",https://www.ihg.com/holidayinn/hotels/us/en/ho...,2901 JOHN HAWKINS PARKWAY,Hoover,Alabama,35244,NaN,4.7
3,Holiday Inn Express & Suites Birmingham - Home...,Holiday Inn Express & Suites Birmingham I-65/...,https://www.ihg.com/holidayinnexpress/hotels/u...,"320 Commons Drive ,","Birmingham,",Alabama,35209,NaN,4.9
4,Holiday Inn Express & Suites Bessemer,Welcome to Holiday Inn Express and Suites Birm...,https://www.ihg.com/holidayinnexpress/hotels/u...,"5001 Academy Lane ,","Bessemer,",Alabama,35022,NaN,4.5
...,...,...,...,...,...,...,...,...,...
3423,Holiday Inn Laramie,The Holiday Inn of Laramie is conveniently loc...,https://www.ihg.com/holidayinn/hotels/us/en/la...,204 30th Street,Laramie,Wyoming,82070,NaN,4.7
3424,Holiday Inn Express Rawlins,"The Holiday Inn Express Rawlins, a Stay Smart ...",https://www.ihg.com/holidayinnexpress/hotels/u...,"201 Airport Road ,","Rawlins,",Wyoming,82301,NaN,4.5
3425,Holiday Inn Express & Suites Rock Springs Gree...,"Hotel's Rock Springs, WY Location Offers Relax...",https://www.ihg.com/holidayinnexpress/hotels/u...,"1660 Sunset Drive ,","Rock Springs,",Wyoming,82901,NaN,4.7
3426,Holiday Inn Rock Springs,An open skyline decorated by White Mountain we...,https://www.ihg.com/holidayinn/hotels/us/en/ro...,1675 Sunset Drive,Rock Springs,Wyoming,82901,NaN,4.5


In [16]:
missing_df['name'].value_counts()

Candlewood Suites Springfield                         2
Candlewood Suites Lafayette                           2
Candlewood Suites Jacksonville                        2
Staybridge Suites St George                           1
Staybridge Suites Sacramento Airport Natomas          1
EVEN Hotels Norwalk                                   1
Candlewood Suites Port Arthur/Nederland               1
Candlewood Suites Dallas, Ft Worth/Fossil Creek       1
Staybridge Suites Denver Tech Center                  1
Candlewood Suites Vestal - Binghamton                 1
Staybridge Suites Alpharetta-North Point              1
Candlewood Suites Memphis - Southaven                 1
Candlewood Suites Charlotte - Arrowood                1
Staybridge Suites Merrillville                        1
EVEN Hotels Eugene                                    1
Candlewood Suites Lodi                                1
Staybridge Suites Coeur d'Alene                       1
Staybridge Suites San Diego-Sorrento Mesa       

In [1]:
hotels_df

NameError: name 'hotels_df' is not defined